## VNPT PERFORMANCE INSIGHTS

#### Các bước cần thực hiện:
 1. Module login 
   - Xây dựng bảng users, users_roles.
   - Thiết kế form login.
   - Xử lý login. 
   - Xử lý robot.
 2. Module insert data
   - Xây dựng bảng data.
   - Thiết kế form insert data.
   - Xử lý insert data.
  3. Module view data
   - Xây dựng form xuất
   - Xử lý biểu đồ
   - Xử lý xuất file
    


In [1]:
import streamlit as st
import pandas as pd
import mysql.connector
from mysql.connector import pooling, Error
from mysql.connector import OperationalError, InternalError
import base64
import datetime
from streamlit_option_menu import option_menu
from datetime import date
import os
import numpy as np
import io
from openpyxl.styles import Font, Border, Side
from openpyxl import Workbook
import openpyxl
import time
import xlsxwriter
import sys
from openpyxl.styles import PatternFill
import requests

### Income and outcome

#### Database connect

In [2]:
def connect_to_mysql():
    try:
        conn = mysql.connector.connect(
            host='103.200.23.68',
            user='samryvnc_anhemtamke',
            password='Tamke@123',
            database='samryvnc_vnpt_performance_insights'
        )
        cursor = conn.cursor()

        print("Kết nối đến cơ sở dữ liệu thành công.")
        
        return conn, cursor
    except Exception as e:
        print("Lỗi khi kết nối đến cơ sở dữ liệu:", e)
        return None, None

In [3]:
# Hàm thực hiện sql trong python
def execute_query_CRUD(conn, query):
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        conn.commit()
        return "Thực hiện thành công"
    except mysql.connector.Error as e:
        return f"Thực hiện thất bại: {e}"

In [10]:
conn,cursor = connect_to_mysql()

Kết nối đến cơ sở dữ liệu thành công.


### 1. INSERT DATA

In [12]:
def query_dichvu_from_database(conn, cursor):
    # đoạn code select
    query = "SELECT ma_dv_id66, ten_dv, danh_muc_tt FROM dichvu"
    cursor.execute(query)
    result = cursor.fetchall()
    ma_dv_66 = [row[0] for row in result]
    ten_dv_66 = [row[1] for row in result]
    danh_muc_tt = [row[2] for row in result]
    # đoạn code tranform
    df_dichvu = pd.DataFrame(list(zip(ma_dv_66, ten_dv_66, danh_muc_tt)), columns =['Mã dịch vụ', 'Tên dịch vụ', 'Danh mục TT'])
    nhom_dv_so_doanh_nghiep = df_dichvu[df_dichvu['Danh mục TT'] == 'Dịch vụ số doanh nghiệp']
    nhom_dv_so_ca_nhan = df_dichvu[df_dichvu['Danh mục TT'] == 'Dịch vụ số cá nhân']
    nhom_dv_vien_thong = df_dichvu[df_dichvu['Danh mục TT'] == 'Dịch vụ Viễn thông']
    nhom_dv_so_doanh_nghiep_id = list(nhom_dv_so_doanh_nghiep['Mã dịch vụ'])
    nhom_dv_so_ca_nhan_id = list(nhom_dv_so_ca_nhan['Mã dịch vụ'])
    nhom_dv_vien_thong_id = list(nhom_dv_vien_thong['Mã dịch vụ'])
    nhom_dv_so_doanh_nghiep_name = list(nhom_dv_so_doanh_nghiep['Tên dịch vụ'])
    nhom_dv_so_ca_nhan_name = list(nhom_dv_so_ca_nhan['Tên dịch vụ'])
    nhom_dv_vien_thong_name = list(nhom_dv_vien_thong['Tên dịch vụ'])
    return nhom_dv_so_doanh_nghiep_id, nhom_dv_so_ca_nhan_id, nhom_dv_vien_thong_id, nhom_dv_so_doanh_nghiep_name, nhom_dv_so_ca_nhan_name, nhom_dv_vien_thong_name

In [22]:
nhom_dv_so_doanh_nghiep_id, nhom_dv_so_ca_nhan_id, nhom_dv_vien_thong_id, nhom_dv_so_doanh_nghiep_name, nhom_dv_so_ca_nhan_name, nhom_dv_vien_thong_name = query_dichvu_from_database(conn,cursor)

#### 1.1 Part cho kế hoạch

In [158]:
def select_rows_kehoach_for_insert(data, nhom_dv_so_doanh_nghiep_id, nhom_dv_so_ca_nhan_id, nhom_dv_vien_thong_id):
    """
    Lọc các dòng từ DataFrame `data` với điều kiện giá trị trong cột `id_dv_606` nằm trong `nhom_dv_id`.
    """
    nhom_dv_id = pd.concat([pd.DataFrame(nhom_dv_so_doanh_nghiep_id), pd.DataFrame(nhom_dv_so_ca_nhan_id), pd.DataFrame(nhom_dv_vien_thong_id)])
    nhom_dv_id = nhom_dv_id.rename(columns={0: 'id_dv'})
    if 'id_dv_606' not in data.columns:
        raise ValueError("Cột 'id_dv_606' không tồn tại trong DataFrame.")

    
    if isinstance(nhom_dv_id, pd.DataFrame):
        nhom_dv_id = nhom_dv_id['id_dv'].tolist()  
    
   
    nhom_dv_id_clean = list(set(nhom_dv_id))
    
    
    subset = data[data['id_dv_606'].isin(nhom_dv_id_clean)]

    return subset

In [159]:
data_insert_kehoach = pd.read_excel('../data/input_kehoach.xlsx')

In [160]:
subset_df = select_rows_kehoach_for_insert(data_insert_kehoach,  nhom_dv_so_doanh_nghiep_id, nhom_dv_so_ca_nhan_id, nhom_dv_vien_thong_id)

In [ ]:
def insertData_kehoach_to_database(line, year,loaidoanhthu, data,conn, cursor):
# Lặp qua từng hàng trong DataFrame
    for index, row in data.iterrows():
        # Câu lệnh SQL để insert dữ liệu
        sql = """INSERT INTO kehoach
                (ma_nv,id_dv_606,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,line,year_insert,loaidoanhthu)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
        # Dữ liệu để insert, kết hợp với line và year
        data_to_insert = tuple(list(row[:14]) + [line, year,loaidoanhthu])
        
        # Thực thi câu lệnh SQL
        cursor.execute(sql, data_to_insert)
    
    # Commit các thay đổi vào cơ sở dữ liệu
    conn.commit()
    return True

In [ ]:
year =2024
line_insert ="Line Cq Nhà Nước, Cơ Quan Hcsn"
loaidoanhthu = "Phát triển mới"
status_insert = insertData_kehoach_to_database(line_insert, year, subset_df, conn, cursor)
if status_insert == True:
    print("Insert dữ liệu vào bảng kehoach thành công")
else:
    print("Insert dữ liệu vào bảng kehoach thất bại")

Insert dữ liệu vào bảng kehoach thành công


#### 1.2 Part cho thực hiện

In [5]:
def group_data_for_insert_thuchien(data):
    grouped_data = data.groupby(['MA_NV', 'NHOM DV']).agg({'DOANH THU': 'sum'}).reset_index()
    return grouped_data

In [36]:
df_thuchien = pd.read_excel('../data/input_thuchien.xlsx')

In [37]:
grouped_thuchien = group_data_for_insert_thuchien(df_thuchien)
print(grouped_thuchien)

        MA_NV                NHOM DV     DOANH THU
0   CTV080832              Băng rộng  6.838078e+07
1   CTV080832         Chính quyền số  2.617000e+06
2   CTV080832                Cố định  2.561463e+07
3   CTV080832        Di động trả sau  3.429502e+07
4   CTV080832      Dịch vụ GTGT khác  1.429793e+06
5   CTV080832         Gói GD,VP,Home  4.990293e+06
6   CTV080832           Hạ tầng CNTT  8.442430e+05
7   CTV080832     Internet trực tiếp  2.729082e+07
8   CTV080832                   MyTV  3.406270e+05
9   CTV080832  Quản trị doanh nghiệp  1.045708e+07
10  CTV082537              Băng rộng  7.340861e+07
11  CTV082537         Chính quyền số  3.617000e+06
12  CTV082537                Cố định  1.679196e+07
13  CTV082537        Di động trả sau  3.383942e+07
14  CTV082537      Dịch vụ GTGT khác  2.181550e+05
15  CTV082537       Dịch vụ tiện ích  1.260000e+05
16  CTV082537         Gói GD,VP,Home  4.015062e+06
17  CTV082537           Hạ tầng CNTT  8.733330e+05
18  CTV082537                  

In [32]:
def insert_data_to_thuchien(thang, nam,loaidoanhthu,line, group_data,conn, cursor):
    
    # Lặp qua từng hàng trong DataFrame
    for index, row in group_data.iterrows():
        # Câu lệnh SQL để insert dữ liệu
        sql = """INSERT INTO thuchien
                (IDnhanvien, nhom_dv, doanhthu,line, thang, year_insert,loaidoanhthu)
                VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        # Dữ liệu để insert, kết hợp với thang và nam
        data_to_insert = (row.iloc[0], row.iloc[1], row.iloc[2],line, thang, nam,loaidoanhthu)
        
        # Thực thi câu lệnh SQL
        cursor.execute(sql, data_to_insert)

    conn.commit()
    return True

In [39]:
line_insert_thuchien ="Line Cq Nhà Nước, Cơ Quan Hcsn"
year_insert_thuchien = 2024
thang = 5
loaidoanhthu = "Hiện hữu"
status_insert_thuchien = insert_data_to_thuchien(thang, year_insert_thuchien,loaidoanhthu,line_insert_thuchien, grouped_thuchien, conn, cursor)
if status_insert_thuchien == True:
    print("Insert dữ liệu vào bảng thuchien thành công")
else:
    print("Insert dữ liệu vào bảng thuchien thất bại")

Insert dữ liệu vào bảng thuchien thành công


### 2. VIEW DATA

#### 2.1 Part cho Kế hoạch

In [5]:
def create_dataframe_public_test(nhom_dv_so_doanh_nghiep_id, nhom_dv_so_ca_nhan_id, nhom_dv_vien_thong_id, nhom_dv_so_doanh_nghiep_name, nhom_dv_so_ca_nhan_name, nhom_dv_vien_thong_name):

    columns = ["STT", "Dịch vụ"] + [f"Tháng {i:02d}" for i in range(1, 13)] + ["LK năm"]

    data = {
        "STT": ["A", "1", "1.1", *nhom_dv_so_doanh_nghiep_id, "1.2",*nhom_dv_so_ca_nhan_id, "2", "2.1", *nhom_dv_vien_thong_id],
        "Dịch vụ": ["DOANH THU VTCNTT", "Công nghệ thông tin", "Dịch vụ số doanh nghiệp",*nhom_dv_so_doanh_nghiep_name ,"Dịch vụ số cá nhân", *nhom_dv_so_ca_nhan_name , "Dịch vụ Viễn thông", "Dịch vụ di động (5917)", *nhom_dv_vien_thong_name],
    }

    # Tạo DataFrame
    kq_dataframe = pd.DataFrame(data, columns=columns)

    return kq_dataframe

In [24]:
df = create_dataframe_public_test(nhom_dv_so_doanh_nghiep_id, nhom_dv_so_ca_nhan_id, nhom_dv_vien_thong_id, nhom_dv_so_doanh_nghiep_name, nhom_dv_so_ca_nhan_name, nhom_dv_vien_thong_name)

In [ ]:
df

#### 2.2 PART CHO DASHBOARD

In [3]:
def create_dataframe_public():
        columns = ["STT", "Dịch vụ"] + [f"Tháng {i:02d}" for i in range(1, 13)] + ["LK năm"]

        data = {
            "STT": ["A", "1", "1.1", "1.1.1", "1.1.2", "1.1.3", "1.1.4", "1.1.5", "1.1.6", "1.1.7", "1.1.8", "1.1.9", "1.2", "1.2.1", "1.2.2", "2", "2.1", "2.1.1", "2.1.2", "2.1.3", "2.2", "2.3", "2.4", "2.5", "2.6", "2.7"],
            "Dịch vụ": ["DOANH THU VTCNTT", "Công nghệ thông tin", "Dịch vụ số doanh nghiệp", "Hạ tầng CNTT", "Chính quyền số", "Y tế số", "Công nghệ nền tảng", "An toàn thông tin", "Giáo dục số", "Quản trị doanh nghiệp", "Vận tải và Logictic", "Phân phối bán lẻ", "Dịch vụ số cá nhân", "MyTV", "Dịch vụ tiện ích", "Dịch vụ Viễn thông", "Dịch vụ di động (5917)", "Di động trả trước", "Di động trả sau", "Gói GD,VP,Home", "Băng rộng", "Cố định", "Internet trực tiếp", "Truyền số liệu", "Dịch vụ GTGT khác", "Cho thuê Hạ tầng"],
        }

        # Tạo DataFrame
        kq_dataframe = pd.DataFrame(data, columns=columns)

        return kq_dataframe

In [ ]:
kq_dataframe = create_dataframe_public()
kq_dataframe

In [5]:
def create_dataframe_thuchien_tt(thang):
        ke_hoach_col = f"KẾ HOẠCH T{thang:02d}"
        thuc_hien_col = f"THỰC HIỆN T{thang:02d}"
        if thang == 1:
            thuc_hien_col = "THỰC HIỆN T1"
        columns = ["STT", "Dịch vụ", ke_hoach_col, thuc_hien_col, "% THỰC HIỆN", "KỲ TRƯỚC", "% VỚI KỲ TRƯỚC", "+/- VỚI KỲ TRƯỚC", "THỰC HIỆN T01", "% THỰC HIỆN T01", "+ /- VỚI T01"]

        # Dữ liệu
        data = {
            "STT": ["A", "1", "1.1", "1.1.1", "1.1.2", "1.1.3", "1.1.4", "1.1.5", "1.1.6", "1.1.7", "1.1.8", "1.1.9", "1.2", "1.2.1", "1.2.2", "2", "2.1", "2.1.1", "2.1.2", "2.1.3", "2.2", "2.3", "2.4", "2.5", "2.6", "2.7"],
            "Dịch vụ": ["DOANH THU VTCNTT", "Công nghệ thông tin", "Dịch vụ số doanh nghiệp", "Hạ tầng CNTT", "Chính quyền số", "Y tế số", "Công nghệ nền tảng", "An toàn thông tin", "Giáo dục số", "Quản trị doanh nghiệp", "Vận tải và Logictic", "Phân phối bán lẻ", "Dịch vụ số cá nhân", "MyTV", "Dịch vụ tiện ích", "Dịch vụ Viễn thông", "Dịch vụ di động (5917)", "Di động trả trước", "Di động trả sau", "Gói GD,VP,Home", "Băng rộng", "Cố định", "Internet trực tiếp", "Truyền số liệu", "Dịch vụ GTGT khác", "Cho thuê Hạ tầng"],
        }

        # Tạo DataFrame với các giá trị NaN cho các cột còn lại
        # Sử dụng một dict comprehension để tạo ra các cột với giá trị NaN
        for col in columns[2:]:
            data[col] = [None] * len(data["STT"])

        # Tạo DataFrame
        kq_thuchien = pd.DataFrame(data, columns=columns)
        return kq_thuchien, ke_hoach_col, thuc_hien_col

In [ ]:
df_th, kehoach,thuchien = create_dataframe_thuchien_tt(5)
df_th

In [9]:
conn,cursor = connect_to_mysql()

Kết nối đến cơ sở dữ liệu thành công.


In [6]:
def query_to_dataframe(query, conn):
    cursor = conn.cursor(dictionary=True)  # Dùng dictionary=True để trả kết quả dạng dict
    cursor.execute(query)
    data = cursor.fetchall()  # Lấy toàn bộ dữ liệu
    cursor.close()
    return pd.DataFrame(data)

In [7]:
def load_data(conn):
    try:
        thuchien = query_to_dataframe("SELECT * FROM thuchien", conn)
        kehoach = query_to_dataframe("SELECT * FROM kehoach", conn)
        nhanvien = query_to_dataframe("SELECT * FROM nhanvien", conn)
        dichvu = query_to_dataframe("SELECT * FROM dichvu", conn)
        for month in range(1, 13):
            kehoach[f"t{month}"] = kehoach[f"t{month}"] * 1000000
        return thuchien, kehoach, nhanvien, dichvu
    finally:
        conn.close()

In [10]:
thuchien_after_load,kehoach_after_load,nhanvien_after_load, dichvu_after_load = load_data(conn)

In [17]:
def filter_data_thuchien_by_employee(thuchien_after_load,ma_nv,month, year,line_nv,loaidoanhthu):
    if month is not None:
        thuchien_after_filter = thuchien_after_load[(thuchien_after_load["IDnhanvien"] == ma_nv) & 
                                                    (thuchien_after_load["line"] == line_nv) &
                                                    (thuchien_after_load["thang"] == month) &
                                                    (thuchien_after_load["year_insert"] == year) &
                                                    (thuchien_after_load["loaidoanhthu"] == loaidoanhthu)]
        thuchien_after_filter = thuchien_after_filter[["nhom_dv", "doanhthu"]]
        thuchien_after_filter["nhom_dv"] = thuchien_after_filter["nhom_dv"].str.strip()
    else:
        thuchien_after_filter = thuchien_after_load[(thuchien_after_load["IDnhanvien"] == ma_nv) & 
                                                    (thuchien_after_load["line"] == line_nv) &
                                                    (thuchien_after_load["year_insert"] == year) &
                                                    (thuchien_after_load["loaidoanhthu"] == loaidoanhthu)]
        thuchien_after_filter = thuchien_after_filter[["nhom_dv", "doanhthu", "thang","year_insert"]]
        thuchien_after_filter["nhom_dv"] = thuchien_after_filter["nhom_dv"].str.strip()
    return thuchien_after_filter

In [14]:
ma_nv = "CTV080832"
line_nv = "Line Cq Nhà Nước, Cơ Quan Hcsn"
thang = 2
loaidoanhthu = "Phát triển mới"
year = 2024
thuchien_after_filter = filter_data_thuchien_by_employee(thuchien_after_load,ma_nv,thang,year,line_nv,loaidoanhthu)


In [16]:
ma_nv = "CTV080832"
line_nv = "Line Cq Nhà Nước, Cơ Quan Hcsn"
thang = 1
loaidoanhthu = "Phát triển mới"
year = 2024
thuchien_after_filter_01 = filter_data_thuchien_by_employee(thuchien_after_load,ma_nv,thang,year,line_nv,loaidoanhthu)

In [ ]:
thuchien_after_filter

In [ ]:
def filter_kehoach_by_employee(kehoach_after_load, dichvu_after_load,selected_ma_nv,line_nv, selected_month,selected_year,selected_loaidoanhthu):
    if selected_month is not None:
        filtered_kehoach = kehoach_after_load[
            (kehoach_after_load["ma_nv"] == selected_ma_nv) &
            (kehoach_after_load["year_insert"] == int(selected_year)) &
            (kehoach_after_load["line"] == line_nv) &
            (kehoach_after_load["loaidoanhthu"] == selected_loaidoanhthu)
        ].copy()
        filtered_kehoach.loc[:, "ten_dv"] = filtered_kehoach["id_dv_606"].map(dict(zip(dichvu_after_load["ma_dv_id66"], dichvu_after_load["ten_dv"])))
        filtered_kehoach = filtered_kehoach[["ten_dv",f"t{selected_month}"]]
    else:
        filtered_kehoach = kehoach_after_load[
            (kehoach_after_load["ma_nv"] == selected_ma_nv) &
            (kehoach_after_load["year_insert"] == int(selected_year)) &
            (kehoach_after_load["line"] == line_nv) &
            (kehoach_after_load["loaidoanhthu"] == selected_loaidoanhthu)
        ].copy()
        filtered_kehoach.loc[:, "ten_dv"] = filtered_kehoach["id_dv_606"].map(dict(zip(dichvu_after_load["ma_dv_id66"], dichvu_after_load["ten_dv"])))
        filtered_kehoach = filtered_kehoach[["ten_dv", "t1", "t2", "t3", "t4", "t5", "t6", "t7", "t8", "t9", "t10", "t11", "t12"]]
        filtered_kehoach.rename(columns={"year_insert": "Năm"}, inplace=True)
    return filtered_kehoach

In [30]:
ma_nv = "CTV080832"
line_nv = "Line Cq Nhà Nước, Cơ Quan Hcsn"
thang = 2
loaidoanhthu = "Phát triển mới"
year = 2024
kehoach_after_filter = filter_kehoach_by_employee(kehoach_after_load,ma_nv,thang,year,loaidoanhthu)

In [36]:
ma_nv = "CTV080832"
line_nv = "Line Cq Nhà Nước, Cơ Quan Hcsn"
thang = None
loaidoanhthu = "Phát triển mới"
year = 2024
kehoach_after_filter_no_month = filter_kehoach_by_employee(kehoach_after_load, dichvu_after_load,ma_nv,thang,year,loaidoanhthu)

In [ ]:
kehoach_after_filter_no_month

In [ ]:
kehoach_after_filter

In [11]:
def query_dichvu_from_database(dichvu_after_load):
    # đoạn code select
    ma_dv_66 = dichvu_after_load['ma_dv_id66'].tolist()
    ten_dv_66 = dichvu_after_load['ten_dv'].tolist()
    danh_muc_tt = dichvu_after_load['danh_muc_tt'].tolist()
    # đoạn code tranform
    df_dichvu = pd.DataFrame(list(zip(ma_dv_66, ten_dv_66, danh_muc_tt)), columns=['Mã dịch vụ', 'Tên dịch vụ', 'Danh mục TT'])
    nhom_dv_so_doanh_nghiep = df_dichvu[df_dichvu['Danh mục TT'] == 'Dịch vụ số doanh nghiệp']
    nhom_dv_so_ca_nhan = df_dichvu[df_dichvu['Danh mục TT'] == 'Dịch vụ số cá nhân']
    nhom_dv_vien_thong = df_dichvu[df_dichvu['Danh mục TT'] == 'Dịch vụ Viễn thông']
    nhom_dv_so_doanh_nghiep_id = list(nhom_dv_so_doanh_nghiep['Mã dịch vụ'])
    nhom_dv_so_ca_nhan_id = list(nhom_dv_so_ca_nhan['Mã dịch vụ'])
    nhom_dv_vien_thong_id = list(nhom_dv_vien_thong['Mã dịch vụ'])
    nhom_dv_so_doanh_nghiep_name = list(nhom_dv_so_doanh_nghiep['Tên dịch vụ'])
    nhom_dv_so_ca_nhan_name = list(nhom_dv_so_ca_nhan['Tên dịch vụ'])
    nhom_dv_vien_thong_name = list(nhom_dv_vien_thong['Tên dịch vụ'])
    return nhom_dv_so_doanh_nghiep_id, nhom_dv_so_ca_nhan_id, nhom_dv_vien_thong_id, nhom_dv_so_doanh_nghiep_name, nhom_dv_so_ca_nhan_name, nhom_dv_vien_thong_name

In [12]:
nhom_dv_so_doanh_nghiep_id, nhom_dv_so_ca_nhan_id, nhom_dv_vien_thong_id, nhom_dv_so_doanh_nghiep_name, nhom_dv_so_ca_nhan_name, nhom_dv_vien_thong_name = query_dichvu_from_database(dichvu_after_load)

In [ ]:
nhom_dv_so_doanh_nghiep_id

In [33]:
def create_dataframe_thuchien_tt(thang,nhom_dv_so_doanh_nghiep_id, nhom_dv_so_ca_nhan_id, nhom_dv_vien_thong_id, nhom_dv_so_doanh_nghiep_name, nhom_dv_so_ca_nhan_name, nhom_dv_vien_thong_name):
    ke_hoach_col = f"KẾ HOẠCH T{thang:02d}"
    thuc_hien_col = f"THỰC HIỆN T{thang:02d}"
    if thang == 1:
        thuc_hien_col = "THỰC HIỆN T1"
    columns = ["STT", "Dịch vụ", ke_hoach_col, thuc_hien_col, "% THỰC HIỆN", "KỲ TRƯỚC", "% VỚI KỲ TRƯỚC", "+/- VỚI KỲ TRƯỚC", "THỰC HIỆN T01", "% THỰC HIỆN T01", "+ /- VỚI T01"]

    # Dữ liệu
    data = {
        "STT": ["A", "1", "1.1", *nhom_dv_so_doanh_nghiep_id, "1.2", *nhom_dv_so_ca_nhan_id, "2", "2.1", *nhom_dv_vien_thong_id],
        "Dịch vụ": ["DOANH THU VTCNTT", "Công nghệ thông tin", "Dịch vụ số doanh nghiệp", *nhom_dv_so_doanh_nghiep_name, "Dịch vụ số cá nhân", *nhom_dv_so_ca_nhan_name, "Dịch vụ Viễn thông", "Dịch vụ di động (5917)", *nhom_dv_vien_thong_name],
    }

    for col in columns[2:]:
        data[col] = [None] * len(data["STT"])

    # Tạo DataFrame
    kq_thuchien = pd.DataFrame(data, columns=columns)
    return kq_thuchien, ke_hoach_col, thuc_hien_col

In [58]:
thang = 2
df_new_create_form_1, kehoach_col_map, thuchien_col_map = create_dataframe_thuchien_tt(thang,nhom_dv_so_doanh_nghiep_id, nhom_dv_so_ca_nhan_id, nhom_dv_vien_thong_id, nhom_dv_so_doanh_nghiep_name, nhom_dv_so_ca_nhan_name, nhom_dv_vien_thong_name)

In [ ]:
df_new_create_form_1

In [47]:
def map_kehoach_to_thuchien_tt(kehoach_after_filter,df_new_create_form_1,ke_hoach_col, thang):
        for index, row in kehoach_after_filter.iterrows():
            service = row["ten_dv"]

            # Tìm vị trí của dịch vụ trong df_new_create_form_1
            matching_rows = df_new_create_form_1[df_new_create_form_1["Dịch vụ"] == service]

            # Nếu có ít nhất một hàng khớp, ánh xạ dữ liệu
            if not matching_rows.empty:
                matching_index = matching_rows.index[0]
                df_new_create_form_1.loc[matching_index, ke_hoach_col] = row[f"t{thang}"] / 1000000
            else:
                print(f"Dịch vụ '{service}' không được tìm thấy trong DataFrame 'df_new_create_form_1'.")
        return df_new_create_form_1

In [60]:
df_new_create_form_1 = map_kehoach_to_thuchien_tt(kehoach_after_filter,df_new_create_form_1,kehoach_col_map,thang)

In [ ]:
df_new_create_form_1

In [62]:
def map_thuchien_tt_for_form(column_data, kq_thuchien_tt, kq_thuchien):
    for index, row in kq_thuchien_tt.iterrows():
        service = row['nhom_dv']

        # Tìm vị trí của dịch vụ trong kq_thuchien
        matching_rows = kq_thuchien[kq_thuchien["Dịch vụ"] == service]

        # Nếu có ít nhất một hàng khớp, ánh xạ dữ liệu
        if not matching_rows.empty:
            matching_index = matching_rows.index[0]
            # Cập nhật giá trị tương ứng trong cột
            column_name = column_data.name
            kq_thuchien.at[matching_index, column_name] = row.iloc[1] / 1000000
        else:
            print(f"Dịch vụ '{service}' không được tìm thấy trong DataFrame 'kq_thuchien'.")
    
    return kq_thuchien

In [63]:
df_new_create_form_1 = map_thuchien_tt_for_form(df_new_create_form_1[thuchien_col_map],thuchien_after_filter,df_new_create_form_1)

In [ ]:
df_new_create_form_1

In [65]:
df_new_create_form_1 = map_thuchien_tt_for_form(df_new_create_form_1["THỰC HIỆN T01"],thuchien_after_filter_01,df_new_create_form_1)

In [67]:
df_new_create_form_1 = map_thuchien_tt_for_form(df_new_create_form_1["KỲ TRƯỚC"],thuchien_after_filter_01,df_new_create_form_1)

In [ ]:
df_new_create_form_1

In [25]:
def update_summary_rows(df):
    df['STT'] = df['STT'].astype(str)

    columns_to_sum = df.columns[2:]

    def get_child_rows(df, parent_stt):
        return df[df['STT'].str.startswith(parent_stt + '.')]

    for index, row in df.iterrows():
        if pd.isna(row[columns_to_sum]).all() or (row[columns_to_sum] == 0).all():

            child_rows = get_child_rows(df, row['STT'])

            if not child_rows.empty:
                df.loc[index, columns_to_sum] = child_rows[columns_to_sum].sum()

    for index, row in df.iterrows():
        if row['STT'] == 'A':
            row_1 = df[df['STT'] == '1'].iloc[:, 2:].sum()
            row_2 = df[df['STT'] == '2'].iloc[:, 2:].sum()

            df.loc[index, columns_to_sum] = row_1 + row_2

    return df

In [188]:
def update_form_1_table(kq_dataframe, thuc_hien_col, ke_hoach_col):
    
    kq_dataframe = kq_dataframe.fillna(0).infer_objects(copy=False)
    kq_dataframe['% THỰC HIỆN'] = np.where(kq_dataframe[ke_hoach_col] != 0,
                                        kq_dataframe[thuc_hien_col] / kq_dataframe[ke_hoach_col] * 100,
                                        0)


    kq_dataframe['% VỚI KỲ TRƯỚC'] = np.where(kq_dataframe['KỲ TRƯỚC'] != 0,
                                            kq_dataframe[thuc_hien_col] / kq_dataframe['KỲ TRƯỚC'] * 100,
                                            0)
    kq_dataframe['+/- VỚI KỲ TRƯỚC'] = kq_dataframe[thuc_hien_col] - kq_dataframe['KỲ TRƯỚC']

    kq_dataframe['% THỰC HIỆN T01'] = np.where(kq_dataframe['THỰC HIỆN T01'] != 0,
                                            kq_dataframe[thuc_hien_col] / kq_dataframe['THỰC HIỆN T01'] * 100,
                                            0)

    kq_dataframe['+ /- VỚI T01'] = kq_dataframe[thuc_hien_col] - kq_dataframe['THỰC HIỆN T01']


    return kq_dataframe

In [ ]:
df_new_create_form_1 = update_summary_rows(df_new_create_form_1)
df_new_create_form_1 = update_form_1_table(df_new_create_form_1,thuchien_col_map, kehoach_col_map)

In [ ]:
df_new_create_form_1

#### FORM 2

In [49]:
ma_nv = "CTV080832"
line_nv = "Line Cq Nhà Nước, Cơ Quan Hcsn"
thang = None
loaidoanhthu = "Phát triển mới"
year = 2024
thuchien_after_filter = filter_data_thuchien_by_employee(thuchien_after_load,ma_nv,thang,year,line_nv,loaidoanhthu)

In [ ]:
thuchien_after_filter

In [79]:
def create_dataframe_form_2(nhom_dv_so_doanh_nghiep_id, nhom_dv_so_ca_nhan_id, nhom_dv_vien_thong_id, nhom_dv_so_doanh_nghiep_name, nhom_dv_so_ca_nhan_name, nhom_dv_vien_thong_name):
    columns = ["STT", "Dịch vụ"] + [f"Tháng {i:02d}" for i in range(1, 13)]

    data = {
        "STT": ["A", "1", "1.1",*nhom_dv_so_doanh_nghiep_id, "1.2", *nhom_dv_so_ca_nhan_id, "2", "2.1", *nhom_dv_vien_thong_id],
        "Dịch vụ": ["DOANH THU VTCNTT", "Công nghệ thông tin", "Dịch vụ số doanh nghiệp",*nhom_dv_so_doanh_nghiep_name, "Dịch vụ số cá nhân", *nhom_dv_so_ca_nhan_name, "Dịch vụ Viễn thông", "Dịch vụ di động (5917)", *nhom_dv_vien_thong_name],
    }

    # Tạo DataFrame
    kq_dataframe = pd.DataFrame(data, columns=columns)

    return kq_dataframe

In [14]:
df_new_create_form_2 = create_dataframe_form_2(nhom_dv_so_doanh_nghiep_id, nhom_dv_so_ca_nhan_id, nhom_dv_vien_thong_id, nhom_dv_so_doanh_nghiep_name, nhom_dv_so_ca_nhan_name, nhom_dv_vien_thong_name)

In [ ]:
df_new_create_form_2

In [20]:
def map_thuchien_to_form_2(kq_thuchien, kq_dataframe):
    for index, row in kq_thuchien.iterrows():
        service = row['nhom_dv']
        month = row['thang']
        revenue = row['doanhthu']

        matching_rows = kq_dataframe[kq_dataframe["Dịch vụ"] == service]

        if not matching_rows.empty:
            matching_index = matching_rows.index[0]
            kq_dataframe.at[matching_index, f"Tháng {int(month):02d}"] = revenue/1000000
        else:
            print(f"Dịch vụ '{service}' không được tìm thấy trong DataFrame 'kq_dataframe'.")
    
    return kq_dataframe

In [22]:
df_new_create_form_2 = map_thuchien_to_form_2(thuchien_after_filter, df_new_create_form_2)

In [ ]:
df_new_create_form_2

In [26]:
df_new_create_form_2 = update_summary_rows(df_new_create_form_2)

In [ ]:
df_new_create_form_2

#### FORM 3

In [88]:
ma_nv = "CTV080832"
line_nv = "Line Cq Nhà Nước, Cơ Quan Hcsn"
thang = None
loaidoanhthu = "Phát triển mới"
year = 2024
kehoach_after_filter_no_month = filter_kehoach_by_employee(kehoach_after_load, dichvu_after_load,ma_nv,thang,year,loaidoanhthu)

In [ ]:
kehoach_after_filter_no_month

In [89]:
form_3_kehoach = create_dataframe_form_2(nhom_dv_so_doanh_nghiep_id, nhom_dv_so_ca_nhan_id, nhom_dv_vien_thong_id, nhom_dv_so_doanh_nghiep_name, nhom_dv_so_ca_nhan_name, nhom_dv_vien_thong_name)

In [ ]:
form_3_kehoach

In [90]:
def map_kehoach_to_form_2(kq_kehoach, kq_dataframe):
        for index, row in kq_kehoach.iterrows():
            service = row["ten_dv"]

 
            matching_rows = kq_dataframe[kq_dataframe["Dịch vụ"] == service]

            if not matching_rows.empty:
                matching_index = matching_rows.index[0]
                kq_dataframe.iloc[matching_index, 2:] = row[1:].values / 1000000
            else:
                print(f"Dịch vụ '{service}' không được tìm thấy trong DataFrame 'kq_dataframe'.")

        return kq_dataframe

In [91]:
kehoach_after_filter_no_month = map_kehoach_to_form_2(kehoach_after_filter_no_month, form_3_kehoach)

In [92]:
kehoach_after_filter_no_month = update_summary_rows(kehoach_after_filter_no_month)

In [ ]:
kehoach_after_filter_no_month

#### SERVICES

In [100]:
def filter_kehoach_by_service(kehoach_after_load,nhanvien_after_load,line_nv, selected_service,selected_month,year_insert,loaidoanhthu):
    if selected_month is not None:
        filtered_kehoach = kehoach_after_load[
            (kehoach_after_load["id_dv_606"] == selected_service) &
            (kehoach_after_load["year_insert"] == year_insert) &
            (kehoach_after_load["line"] == line_nv) &
            (kehoach_after_load["loaidoanhthu"] == loaidoanhthu)
        ].copy()
        filtered_kehoach.loc[:, "ten_nv"] = filtered_kehoach["ma_nv"].map(dict(zip(nhanvien_after_load["ma_nv"], nhanvien_after_load["ten_nv"])))
        filtered_kehoach = filtered_kehoach[["ma_nv","ten_nv",f"t{selected_month}"]]
    else:
        filtered_kehoach = kehoach_after_load[
            (kehoach_after_load["id_dv_606"] == selected_service) &
            (kehoach_after_load["year_insert"] == year_insert) &
            (kehoach_after_load["line"] == line_nv) &
            (kehoach_after_load["loaidoanhthu"] == loaidoanhthu)
        ].copy()
        filtered_kehoach.loc[:, "ten_nv"] = filtered_kehoach["ma_nv"].map(dict(zip(nhanvien_after_load["ma_nv"], nhanvien_after_load["ten_nv"])))
        filtered_kehoach = filtered_kehoach[["ma_nv","ten_nv", "t1", "t2", "t3", "t4", "t5", "t6", "t7", "t8", "t9", "t10", "t11", "t12"]]
        filtered_kehoach.rename(columns={"year_insert": "Năm"}, inplace=True)
    return filtered_kehoach
   

In [101]:
ma_dv = "1.1.01"
line_nv = "Line Cq Nhà Nước, Cơ Quan Hcsn"
thang = None
loaidoanhthu = "Phát triển mới"
year = 2024
kehoach_after_filter_no_month = filter_kehoach_by_service(kehoach_after_load,nhanvien_after_load,line_nv,ma_dv,thang,year,loaidoanhthu)

In [154]:
ma_dv = "2.2"
line_nv = "Line Cq Nhà Nước, Cơ Quan Hcsn"
thang = 2
loaidoanhthu = "Phát triển mới"
year = 2024
kehoach_after_filter = filter_kehoach_by_service(kehoach_after_load,nhanvien_after_load,line_nv,ma_dv,thang,year,loaidoanhthu)

In [155]:
kehoach_after_filter

,ma_nv,ten_nv,t2
51,CTV080832,Trần Thanh Liêm,3760000.0
70,CTV082537,Nguyễn Thị Hồng Hạnh,0.0


In [ ]:
def filter_thuchien_by_service(thuchien_after_load,ten_dv,month, year,line_nv,loaidoanhthu):
    if month is not None:
        thuchien_after_filter = thuchien_after_load[(thuchien_after_load["nhom_dv"] == ten_dv) & 
                                                    (thuchien_after_load["line"] == line_nv) &
                                                    (thuchien_after_load["thang"] == month) &
                                                    (thuchien_after_load["year_insert"] == year) &
                                                    (thuchien_after_load["loaidoanhthu"] == loaidoanhthu)]
        thuchien_after_filter = thuchien_after_filter[["IDnhanvien", "doanhthu"]]
        thuchien_after_filter["IDnhanvien"] = thuchien_after_filter["IDnhanvien"].str.strip()
    else:
        thuchien_after_filter = thuchien_after_load[(thuchien_after_load["nhom_dv"] == ten_dv) & 
                                                    (thuchien_after_load["line"] == line_nv) &
                                                    (thuchien_after_load["year_insert"] == year) &
                                                    (thuchien_after_load["loaidoanhthu"] == loaidoanhthu)]
        thuchien_after_filter = thuchien_after_filter[["IDnhanvien", "doanhthu", "thang","year_insert"]]
        thuchien_after_filter["IDnhanvien"] = thuchien_after_filter["IDnhanvien"].str.strip()
    return thuchien_after_filter

In [152]:
ten_dv = "Băng rộng"
line_nv = "Line Cq Nhà Nước, Cơ Quan Hcsn"
thang = 2
loaidoanhthu = "Phát triển mới"
year = 2024
thuchien_after_filter = filter_thuchien_by_service(thuchien_after_load,ten_dv,thang,year,line_nv,loaidoanhthu)

In [153]:
thuchien_after_filter

,IDnhanvien,doanhthu
21,CTV080832,66812800.0
31,CTV082537,71728600.0


In [159]:
ten_dv = "Băng rộng"
line_nv = "Line Cq Nhà Nước, Cơ Quan Hcsn"
thang = 1
loaidoanhthu = "Phát triển mới"
year = 2024
thuchien_after_filter_01 = filter_thuchien_by_service(thuchien_after_load,ten_dv,thang,year,line_nv,loaidoanhthu)

In [160]:
thuchien_after_filter_01

,IDnhanvien,doanhthu
0,CTV080832,66714800.0
10,CTV082537,71623600.0


In [110]:
def create_dataframe_form_4(nhanvien_after_load):
    columns = ["STT", "Nhân viên"] + [f"Tháng {i:02d}" for i in range(1, 13)]

    data = {
        "STT": ["A", *nhanvien_after_load["ma_nv"].tolist()],
        "Nhân viên": ["Tổng cộng"] + nhanvien_after_load["ten_nv"].tolist(),
    }

    for col in columns[2:]:
        data[col] = [None] * len(data["STT"])

    # Tạo DataFrame
    kq_dataframe = pd.DataFrame(data, columns=columns)

    return kq_dataframe

In [112]:
data_frame_4 = create_dataframe_form_4(nhanvien_after_load)

In [122]:
def create_thuchien_service_dataframe_form_1(thang,nhanvien_after_load):
    kehoach_col = f"KẾ HOẠCH T{thang:02d}"
    thuchien_col = f"THỰC HIỆN T{thang:02d}"
    if thang == 1:
        thuchien_col = "THỰC HIỆN T1"
    columns = ["MA-NV", "Nhân viên", kehoach_col, thuchien_col, "% THỰC HIỆN", "KỲ TRƯỚC", "% VỚI KỲ TRƯỚC", "+/- VỚI KỲ TRƯỚC", "THỰC HIỆN T01", "% THỰC HIỆN T01", "+ /- VỚI T01"]
    
    data = {
        "MA-NV": ["A", *nhanvien_after_load["ma_nv"].tolist()],
        "Nhân viên": ["Tổng cộng"] + nhanvien_after_load["ten_nv"].tolist(),
    }
    for col in columns[2:]:
        data[col] = [None] * len(data["MA-NV"])
    kq_thuchien = pd.DataFrame(data, columns=columns)
    return kq_thuchien, kehoach_col, thuchien_col

In [156]:
thang = 2
df_create_service_form_1, kehoach_col_map, thuchien_col_map = create_thuchien_service_dataframe_form_1(thang,nhanvien_after_load)

In [147]:
df_create_service_form_1

,MA-NV,Nhân viên,KẾ HOẠCH T02,THỰC HIỆN T02,% THỰC HIỆN,KỲ TRƯỚC,% VỚI KỲ TRƯỚC,+/- VỚI KỲ TRƯỚC,THỰC HIỆN T01,% THỰC HIỆN T01,+ /- VỚI T01
0,A,Tổng cộng,None,None,None,None,None,None,None,None,None
1,CTV080832,Trần Thanh Liêm,None,None,None,None,None,None,None,None,None
2,CTV082537,Nguyễn Thị Hồng Hạnh,None,None,None,None,None,None,None,None,None


In [148]:
def map_kehoach_to_thuchien_service(kehoach_after_filter,df_new_create_form_1,ke_hoach_col, thang):
        for index, row in kehoach_after_filter.iterrows():
            employee = row["ma_nv"]

            # Tìm vị trí của dịch vụ trong df_new_create_form_1
            matching_rows = df_new_create_form_1[df_new_create_form_1["MA-NV"] == employee]

            # Nếu có ít nhất một hàng khớp, ánh xạ dữ liệu
            if not matching_rows.empty:
                matching_index = matching_rows.index[0]
                df_new_create_form_1.loc[matching_index, ke_hoach_col] = row[f"t{thang}"] / 1000000
            else:
                print(f"Nhan vien '{employee}' không được tìm thấy trong DataFrame 'df_new_create_form_1'.")
        return df_new_create_form_1

In [157]:
df_create_service_form_1 = map_kehoach_to_thuchien_service(kehoach_after_filter,df_create_service_form_1,kehoach_col_map,thang)

In [158]:
df_create_service_form_1

,MA-NV,Nhân viên,KẾ HOẠCH T02,THỰC HIỆN T02,% THỰC HIỆN,KỲ TRƯỚC,% VỚI KỲ TRƯỚC,+/- VỚI KỲ TRƯỚC,THỰC HIỆN T01,% THỰC HIỆN T01,+ /- VỚI T01
0,A,Tổng cộng,None,None,None,None,None,None,None,None,None
1,CTV080832,Trần Thanh Liêm,3.76,None,None,None,None,None,None,None,None
2,CTV082537,Nguyễn Thị Hồng Hạnh,0.0,None,None,None,None,None,None,None,None


In [163]:
def map_thuchien_service_for_form(column_data, kq_thuchien_tt, kq_thuchien):
    for index, row in kq_thuchien_tt.iterrows():
        employee = row['IDnhanvien']

        # Tìm vị trí của dịch vụ trong kq_thuchien
        matching_rows = kq_thuchien[kq_thuchien["MA-NV"] == employee]

        # Nếu có ít nhất một hàng khớp, ánh xạ dữ liệu
        if not matching_rows.empty:
            matching_index = matching_rows.index[0]
            # Cập nhật giá trị tương ứng trong cột
            column_name = column_data.name
            kq_thuchien.at[matching_index, column_name] = row.iloc[1] / 1000000
        else:
            print(f"Nhan vien '{employee}' không được tìm thấy trong DataFrame 'kq_thuchien'.")
    
    return kq_thuchien

In [164]:
df_create_service_form_1 = map_thuchien_service_for_form(df_create_service_form_1[thuchien_col_map],thuchien_after_filter,df_create_service_form_1)

In [166]:
df_create_service_form_1 = map_thuchien_service_for_form(df_create_service_form_1["THỰC HIỆN T01"],thuchien_after_filter_01,df_create_service_form_1)

In [167]:
df_create_service_form_1 = map_thuchien_service_for_form(df_create_service_form_1["KỲ TRƯỚC"],thuchien_after_filter_01,df_create_service_form_1)

In [168]:
df_create_service_form_1

,MA-NV,Nhân viên,KẾ HOẠCH T02,THỰC HIỆN T02,% THỰC HIỆN,KỲ TRƯỚC,% VỚI KỲ TRƯỚC,+/- VỚI KỲ TRƯỚC,THỰC HIỆN T01,% THỰC HIỆN T01,+ /- VỚI T01
0,A,Tổng cộng,None,None,None,None,None,None,None,None,None
1,CTV080832,Trần Thanh Liêm,3.76,66.8128,None,66.7148,None,None,66.7148,None,None
2,CTV082537,Nguyễn Thị Hồng Hạnh,0.0,71.7286,None,71.6236,None,None,71.6236,None,None


In [ ]:
def update_summary_rows_service(df):
    numeric_cols = df.columns[2:]  

    sums = df.loc[1:, numeric_cols].sum()

    df.loc[0, numeric_cols] = sums

    return df

In [185]:
df_create_service_form_1 = update_summary_rows_service(df_create_service_form_1)

In [189]:
df_create_service_form_1 = update_form_1_table(df_create_service_form_1,thuchien_col_map, kehoach_col_map)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_37432\203376480.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  kq_dataframe = kq_dataframe.fillna(0).infer_objects(copy=False)


In [190]:
df_create_service_form_1

,MA-NV,Nhân viên,KẾ HOẠCH T02,THỰC HIỆN T02,% THỰC HIỆN,KỲ TRƯỚC,% VỚI KỲ TRƯỚC,+/- VỚI KỲ TRƯỚC,THỰC HIỆN T01,% THỰC HIỆN T01,+ /- VỚI T01
0,A,Tổng cộng,3.76,138.5414,3684.611702,138.3384,100.146742,0.203,138.3384,100.146742,0.203
1,CTV080832,Trần Thanh Liêm,3.76,66.8128,1776.936170,66.7148,100.146894,0.098,66.7148,100.146894,0.098
2,CTV082537,Nguyễn Thị Hồng Hạnh,0.00,71.7286,0.000000,71.6236,100.146600,0.105,71.6236,100.146600,0.105


#### FORM 2 SERVICES

In [191]:
ten_dv = "Băng rộng"
line_nv = "Line Cq Nhà Nước, Cơ Quan Hcsn"
thang = None
loaidoanhthu = "Phát triển mới"
year = 2024
thuchien_after_filter = filter_thuchien_by_service(thuchien_after_load,ten_dv,thang,year,line_nv,loaidoanhthu)

In [192]:
thuchien_after_filter

,IDnhanvien,doanhthu,thang,year_insert
0,CTV080832,66714800.0,1,2024
10,CTV082537,71623600.0,1,2024
21,CTV080832,66812800.0,2,2024
31,CTV082537,71728600.0,2,2024
42,CTV080832,66616800.0,3,2024
52,CTV082537,71518600.0,3,2024


In [193]:
def create_data_service_form_2(nhanvien_after_load):
    columns = ["MA-NV", "Nhân viên"] + [f"Tháng {i:02d}" for i in range(1, 13)]

    data = {
        "MA-NV": ["A", *nhanvien_after_load["ma_nv"].tolist()],
        "Nhân viên": ["Tổng cộng"] + nhanvien_after_load["ten_nv"].tolist(),
    }

    for col in columns[2:]:
        data[col] = [None] * len(data["MA-NV"])

    # Tạo DataFrame
    kq_dataframe = pd.DataFrame(data, columns=columns)

    return kq_dataframe

In [194]:
form_2_df_service = create_data_service_form_2(nhanvien_after_load)

In [195]:
form_2_df_service

,MA-NV,Nhân viên,Tháng 01,Tháng 02,Tháng 03,Tháng 04,Tháng 05,Tháng 06,Tháng 07,Tháng 08,Tháng 09,Tháng 10,Tháng 11,Tháng 12
0,A,Tổng cộng,None,None,None,None,None,None,None,None,None,None,None,None
1,CTV080832,Trần Thanh Liêm,None,None,None,None,None,None,None,None,None,None,None,None
2,CTV082537,Nguyễn Thị Hồng Hạnh,None,None,None,None,None,None,None,None,None,None,None,None


In [196]:
def map_thuchien_service_to_form_2(kq_thuchien, kq_dataframe):
    for index, row in kq_thuchien.iterrows():
        service = row['IDnhanvien']
        month = row['thang']
        revenue = row['doanhthu']

        matching_rows = kq_dataframe[kq_dataframe["MA-NV"] == service]

        if not matching_rows.empty:
            matching_index = matching_rows.index[0]
            kq_dataframe.at[matching_index, f"Tháng {int(month):02d}"] = revenue/1000000
        else:
            print(f"MA-NV '{service}' không được tìm thấy trong DataFrame 'kq_dataframe'.")
    
    return kq_dataframe

In [197]:
form_2_df_service= map_thuchien_service_to_form_2(thuchien_after_filter, form_2_df_service)

In [199]:
form_2_df_service = update_summary_rows_service(form_2_df_service)

In [200]:
form_2_df_service

,MA-NV,Nhân viên,Tháng 01,Tháng 02,Tháng 03,Tháng 04,Tháng 05,Tháng 06,Tháng 07,Tháng 08,Tháng 09,Tháng 10,Tháng 11,Tháng 12
0,A,Tổng cộng,138.3384,138.5414,138.1354,0,0,0,0,0,0,0,0,0
1,CTV080832,Trần Thanh Liêm,66.7148,66.8128,66.6168,None,None,None,None,None,None,None,None,None
2,CTV082537,Nguyễn Thị Hồng Hạnh,71.6236,71.7286,71.5186,None,None,None,None,None,None,None,None,None


#### FORM 3 SERVICES

In [205]:
ma_dv = "1.1.01"
line_nv = "Line Cq Nhà Nước, Cơ Quan Hcsn"
thang = None
loaidoanhthu = "Phát triển mới"
year = 2024
kehoach_after_filter_no_month = filter_kehoach_by_service(kehoach_after_load,nhanvien_after_load,line_nv,ma_dv,thang,year,loaidoanhthu)

In [206]:
kehoach_after_filter_no_month

,ma_nv,ten_nv,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12
38,CTV080832,Trần Thanh Liêm,2070000.0,5880000.0,10120000.0,14070000.0,16980000.0,20020000.0,23140000.0,26330000.0,29450000.0,32430000.0,35550000.0,38740000.0
57,CTV082537,Nguyễn Thị Hồng Hạnh,0.0,0.0,0.0,0.0,0.0,0.0,50000.0,50000.0,50000.0,50000.0,50000.0,50000.0


In [207]:
df_form3_service = create_data_service_form_2(nhanvien_after_load)

In [209]:
def map_kehoach_to_form_2(kq_kehoach, kq_dataframe):
        for index, row in kq_kehoach.iterrows():
            employee = row["ma_nv"]
 
            matching_rows = kq_dataframe[kq_dataframe["MA-NV"] == employee]

            if not matching_rows.empty:
                matching_index = matching_rows.index[0]
                kq_dataframe.iloc[matching_index, 2:] = row[2:].values / 1000000
            else:
                print(f"Nhan vien '{employee}' không được tìm thấy trong DataFrame 'kq_dataframe'.")

        return kq_dataframe

In [210]:
df_form3_service = map_kehoach_to_form_2(kehoach_after_filter_no_month, df_form3_service)

In [211]:
df_form3_service

,MA-NV,Nhân viên,Tháng 01,Tháng 02,Tháng 03,Tháng 04,Tháng 05,Tháng 06,Tháng 07,Tháng 08,Tháng 09,Tháng 10,Tháng 11,Tháng 12
0,A,Tổng cộng,None,None,None,None,None,None,None,None,None,None,None,None
1,CTV080832,Trần Thanh Liêm,2.07,5.88,10.12,14.07,16.98,20.02,23.14,26.33,29.45,32.43,35.55,38.74
2,CTV082537,Nguyễn Thị Hồng Hạnh,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05,0.05,0.05,0.05,0.05


In [212]:
df_form3_service = update_summary_rows_service(df_form3_service)

In [213]:
df_form3_service

,MA-NV,Nhân viên,Tháng 01,Tháng 02,Tháng 03,Tháng 04,Tháng 05,Tháng 06,Tháng 07,Tháng 08,Tháng 09,Tháng 10,Tháng 11,Tháng 12
0,A,Tổng cộng,2.07,5.88,10.12,14.07,16.98,20.02,23.19,26.38,29.5,32.48,35.6,38.79
1,CTV080832,Trần Thanh Liêm,2.07,5.88,10.12,14.07,16.98,20.02,23.14,26.33,29.45,32.43,35.55,38.74
2,CTV082537,Nguyễn Thị Hồng Hạnh,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05,0.05,0.05,0.05,0.05


In [6]:
def query_to_dataframe(query, conn):
    cursor = conn.cursor(dictionary=True)  # Dùng dictionary=True để trả kết quả dạng dict
    cursor.execute(query)
    data = cursor.fetchall()  # Lấy toàn bộ dữ liệu
    cursor.close()
    return pd.DataFrame(data)

In [12]:
def load_data(conn):
    try:
        dichvu = query_to_dataframe("SELECT * FROM dichvu", conn)
        line = query_to_dataframe("SELECT * FROM line_manage", conn)
        return dichvu, line
    finally:
        conn.close()

In [81]:
conn,cursor = connect_to_mysql()

Kết nối đến cơ sở dữ liệu thành công.


In [82]:
dichvu, line = load_data(conn)

In [83]:
def build_hierarchy(df):
    if not {'ma_dv_id66', 'ten_dv', 'danh_muc_tt'}.issubset(df.columns):
        raise ValueError("DataFrame phải có các cột 'ma_dv_id66', 'ten_dv', và 'danh_muc_tt'")
    
    df['full_path'] = df['ten_dv']

    def build_full_path(row, df):
        if pd.isna(row['danh_muc_tt']):
            return row['ten_dv']
        else:
            parent_row = df[df['ma_dv_id66'] == row['danh_muc_tt']]
            if not parent_row.empty:
                parent_full_path = parent_row['full_path'].values[0]
                return f"{parent_full_path}.{row['ten_dv']}"
            return row['ten_dv']

    df['full_path'] = df.apply(lambda row: build_full_path(row, df), axis=1)

    df_all = df.sort_values(by=['ma_dv_id66', 'full_path'])
    
    df_filtered = df[df['ma_dv_id66'].str.count('\.') <= 2]
    
    return df_all[['ma_dv_id66', 'ten_dv', 'full_path']], df_filtered[['ma_dv_id66', 'ten_dv', 'full_path']]

In [85]:
def sorted_service_from_db(dichvu):
    df_all_1,df_all_2 = build_hierarchy(dichvu)
    ma_dv_1 = df_all_1['ma_dv_id66'].tolist()
    ten_dv_1 = df_all_2['ten_dv'].tolist()
    ma_dv_2 = df_all_2['ma_dv_id66'].tolist()
    ten_dv_2 = df_all_2['ten_dv'].tolist()
    return ma_dv_1, ten_dv_1, ma_dv_2, ten_dv_2

In [86]:
ma_dv_1, ten_dv_1, ma_dv_2, ten_dv_2 = sorted_service_from_db(dichvu)

In [ ]:
def create_dataframe_thuchien_tt(thang, ma_dv, ten_dv):
    ke_hoach_col = f"KẾ HOẠCH T{thang:02d}"
    thuc_hien_col = f"THỰC HIỆN T{thang:02d}"
    if thang == 1:
        thuc_hien_col = "THỰC HIỆN T1"
    columns = ["STT", "Dịch vụ", ke_hoach_col, thuc_hien_col, "% THỰC HIỆN", "KỲ TRƯỚC", "% VỚI KỲ TRƯỚC", "+/- VỚI KỲ TRƯỚC", "THỰC HIỆN T01", "% THỰC HIỆN T01", "+ /- VỚI T01"]

    # Dữ liệu
    data = {
        "STT": [*ma_dv],
        "Dịch vụ": [*ten_dv],
    }

    for col in columns[2:]:
        data[col] = [None] * len(data["STT"])

    # Tạo DataFrame
    kq_thuchien = pd.DataFrame(data, columns=columns)
    return kq_thuchien, ke_hoach_col, thuc_hien_col

In [88]:
kq_thuchien, ke_hoach_col, thuc_hien_col = create_dataframe_thuchien_tt(2, ma_dv_2, ten_dv_2)

In [90]:
def create_dataframe_form_2(ma_dv, ten_dv):
    columns = ["STT", "Dịch vụ"] + [f"Tháng {i:02d}" for i in range(1, 13)]

    data = {
        "STT": [*ma_dv],
        "Dịch vụ": [*ten_dv],
    }

    # Tạo DataFrame
    kq_dataframe = pd.DataFrame(data, columns=columns)

    return kq_dataframe

In [91]:
kq_dataframe = create_dataframe_form_2(ma_dv_2, ten_dv_2)

In [ ]:
kq_dataframe

In [96]:
df_test = pd.read_excel("../ket_qua_danh_muc_tt.xlsx", sheet_name="Sheet1")

In [97]:
df_test_2 = pd.read_excel("../ket_qua_danh_muc_tt.xlsx", sheet_name="ket_qua_danh_muc_tt")

In [ ]:
df_test_2

In [ ]:
# Merge df_test_2 with df_test to get the corresponding ma_dv_id6 values
df_merged = df_test_2.merge(df_test, left_on='nhom_dv', right_on='ten_dv', how='left')

# Select the relevant columns
df_merged = df_merged[['nhom_dv', 'ma_dv_id6']]



                   nhom_dv ma_dv_id6
0                Băng rộng       2.2
1           Chính quyền số    1.1.02
2                  Cố định       2.3
3          Di động trả sau    2.1.02
4        Dịch vụ GTGT khác       2.6
..                     ...       ...
121         Gói GD,VP,Home    2.1.03
122           Hạ tầng CNTT    1.1.01
123                   MyTV    1.2.01
124  Quản trị doanh nghiệp    1.1.07
125         Truyền số liệu       2.5

[126 rows x 2 columns]


In [102]:
df_merged = pd.DataFrame(df_merged)

In [105]:
df_merged.to_excel("../ket_qua_danh_muc_tt.xlsx", sheet_name="Sheet3", index=False)
